In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import csv 

from preprocessingfun import preprocessingzinc, sigmastd, createincompletedata, dataindices
from miscfun import round_to_poweroften, calculatemse, calculatemaxerror
from othermodels import gaussianmodel, Ridgemodel, MLPmodel
from mainmodel import truncatednormalmodel, samplefromlikelihood
from coveragefun import calculatecoverage

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
dataflows = pd.read_csv("data/zincflowschina.txt")
datastocks = pd.read_csv("data/zincstockchina.txt")

#preprocess the data
datamat,outputdatachildprocess,availablechildstocksandflows,m,N=preprocessingzinc(datastocks,dataflows)

priorcov=16.0
priorcovvague=40.0
alpha=sigmastd/priorcovvague

#construct the prior, uninformative

priormeanvague = np.array([1.0] * N)
covariancevecvague = 40.0  * np.ones(N)

dataavg=np.mean(np.abs(datamat[:,1]))

for row in datamat:
    priormeanvague[int(row[0])]=np.sign(row[1])*dataavg
    covariancevecvague[int(row[0])] = priorcovvague

priormeanvaguecompact = priormeanvague[availablechildstocksandflows]
covariancevecvaguecompact = covariancevecvague[availablechildstocksandflows]
priorcovariancevaguecompact = np.diag(covariancevecvaguecompact)

#construct the weakly informative prior

priormean = np.array([1.0] * N)
covariancevec = 16.0  * np.ones(N)

for row in datamat:
    priormean[int(row[0])]=round_to_poweroften(row[1])
    covariancevec[int(row[0])]=max(min(priorcov,16*round_to_poweroften(row[1])**2),0.01)

priormeancompact = priormean[availablechildstocksandflows]
covarianceveccompact = covariancevec[availablechildstocksandflows]
priorcovariancecompact = np.diag(covarianceveccompact)


In [ ]:
#Calculate the root mean squared error and maximum error of a number of models, specifically
#Proposed model, Gaussian model, Ridge regression, MLP over for each dataset size from 0 to 20,
#averaged over 50 runs.

truevalues=outputdatachildprocess['quantity'][availablechildstocksandflows]
for seed in range(0,50):
    random.seed(seed)
    pointsall = random.sample(list(availablechildstocksandflows), len(availablechildstocksandflows))
    print('seed')
    print(seed)
    for i in range(0,21): 
        points=pointsall[0:i]
        print('i')
        print(i)
        incompletedesigncompact,incompletematrixstockscompact,incompletematrixflowscompact,incompletedatavector,incompletedataflownumber=createincompletedata(points,datamat,availablechildstocksandflows,m,N) 

        regcoef=Ridgemodel(incompletedesigncompact,incompletedatavector,alpha)
        mseridge=calculatemse(regcoef,truevalues)
        maxerrorridge=calculatemaxerror(regcoef,truevalues)
        
        resultsridge=[i,mseridge,maxerrorridge,seed,alpha]

        import csv
        with open("zincridgeerror.txt", "a") as f0:
            wr0 = csv.writer(f0)
            wr0.writerow(resultsridge)
        f0.close()

        posteriormeangauss,posteriorcovariancegauss=gaussianmodel(priormeancompact,priorcovariancecompact,incompletedesigncompact,incompletedatavector)
        posteriormeangaussvague,posteriorcovariancegaussvague=gaussianmodel(priormeanvaguecompact,priorcovariancevaguecompact,incompletedesigncompact,incompletedatavector)
        
        
        msegauss=calculatemse(posteriormeangauss,truevalues)
        maxerrorgauss=calculatemaxerror(posteriormeangauss,truevalues)
        
        msegaussvague=calculatemse(posteriormeangaussvague,truevalues)
        maxerrorgaussvague=calculatemaxerror(posteriormeangaussvague,truevalues)
        
        resultsgauss=[i,msegauss,maxerrorgauss,seed,sigmastd,covariancevec[int(row[0])]]
        resultsgaussvague=[i,msegaussvague,maxerrorgaussvague,seed,sigmastd,covariancevecvague[int(row[0])]]

        import csv
        with open("zincweaklyinformativegaussianerror.txt", "a") as f1:
            wr1 = csv.writer(f1)
            wr1.writerow(resultsgauss)
        f1.close()
        with open("zincuninformativegaussianerror.txt", "a") as f2:
            wr2 = csv.writer(f2)
            wr2.writerow(resultsgaussvague)
        f2.close()

        modelpred,traceoutput=truncatednormalmodel(priormean,covariancevec,incompletematrixstockscompact,incompletematrixflowscompact,incompletedatavector,incompletedataflownumber,availablechildstocksandflows,m,1)
        
        truncatednormalmse=calculatemse(modelpred,truevalues)
        truncatednormalmaxerror=calculatemaxerror(modelpred,truevalues)
        
        resultstruncatednormal=[i,truncatednormalmse,truncatednormalmaxerror,seed,sigmastd,covariancevec[int(row[0])]]

        with open("zincweaklyinformativemodelerror.txt", "a") as f3:
            wr3 = csv.writer(f3)
            wr3.writerow(resultstruncatednormal)
        f3.close()

        modelpredvague,traceoutputvague=truncatednormalmodel(priormeanvague,covariancevecvague,incompletematrixstockscompact,incompletematrixflowscompact,incompletedatavector,incompletedataflownumber,availablechildstocksandflows,m,1)
        
        truncatednormalmsevague=calculatemse(modelpredvague,truevalues)
        truncatednormalmaxerrorvague=calculatemaxerror(modelpredvague,truevalues)
        
        resultstruncatednormalvague=[i,truncatednormalmsevague,truncatednormalmaxerrorvague,seed,sigmastd,covariancevecvague[int(row[0])]]

        with open("zincuninformativemodelerror.txt", "a") as f4:
            wr4 = csv.writer(f4)
            wr4.writerow(resultstruncatednormalvague)
        f4.close()
        
        
        mlppred=MLPmodel(incompletedesigncompact,incompletedatavector)
        
        msemlp=calculatemse(mlppred,truevalues)
        maxerrormlp=calculatemaxerror(mlppred,truevalues)

        resultsmlp=[i,msemlp,maxerrormlp,seed]

        import csv
        with open("zincmlperror.txt", "a") as f5:
            wr5 = csv.writer(f5)
            wr5.writerow(resultsmlp)
        f5.close()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#generate rmse plots of the four models, based on model outputs from previous cell

zincgaussianmse = np.loadtxt("zincweaklyinformativegaussianerror.txt", comments="#", delimiter=",", unpack=False)
zincgaussianmsevague = np.loadtxt("zincuninformativegaussianerror.txt", comments="#", delimiter=",", unpack=False)

zincmapmse = np.loadtxt("zincweaklyinformativemodelerror.txt", comments="#", delimiter=",", unpack=False)
zincmapmsevague = np.loadtxt("zincuninformativemodelerror.txt", comments="#", delimiter=",", unpack=False)

zincfreqmse = np.loadtxt("zincridgeerror.txt", comments="#", delimiter=",", unpack=False)

zincmlpmse=np.loadtxt("zincmlperror.txt", comments="#", delimiter=",", unpack=False)

zincgaussianmsedf = pd.DataFrame(dict(n=zincgaussianmse[:,0], l2error=zincgaussianmse[:,1]))

zincgaussianmsevaguedf = pd.DataFrame(dict(n=zincgaussianmsevague[:,0], l2error=zincgaussianmsevague[:,1]))

zincmapmsedf = pd.DataFrame(dict(n=zincmapmse[:,0], l2error=zincmapmse[:,1]))

zincmapmsevaguedf = pd.DataFrame(dict(n=zincmapmsevague[:,0], l2error=zincmapmsevague[:,1]))

zincfreqmsedf = pd.DataFrame(dict(n=zincfreqmse[:,0], l2error=zincfreqmse[:,1]))

zincmlpmsedf = pd.DataFrame(dict(n=zincmlpmse[:,0], l2error=zincmlpmse[:,1]))

plt.rcParams.update({
    "text.usetex": True,})

zincgaussianmsedfmean=zincgaussianmsedf.groupby(['n'],as_index=False).mean()
zincgaussianmsevaguedfmean=zincgaussianmsevaguedf.groupby(['n'],as_index=False).mean()
zincmapmsedfmean=zincmapmsedf.groupby(['n'],as_index=False).mean()
zincmapmsevaguedfmean=zincmapmsevaguedf.groupby(['n'],as_index=False).mean()

zincfreqmsedfmean=zincfreqmsedf.groupby(['n'],as_index=False).mean()

zincmlpmsedfmean=zincmlpmsedf.groupby(['n'],as_index=False).mean()

plt.xticks(range(0,21)) #ensures integer values on x axis. 

plt.plot(zincgaussianmsedfmean['n'], zincgaussianmsedfmean['l2error'],label='Informative Prior, Gaussian Model')
plt.plot(zincgaussianmsevaguedfmean['n'],zincgaussianmsevaguedfmean['l2error'],label='Vague Prior, Gaussian Model')

plt.plot(zincmapmsedfmean['n'], zincmapmsedfmean['l2error'],label='Informative Prior, Proposed Model')
plt.plot(zincmapmsevaguedfmean['n'], zincmapmsevaguedfmean['l2error'],label='Vague Prior, Proposed Model')
plt.plot(zincfreqmsedfmean['n'], zincfreqmsedfmean['l2error'],label='Ridge Regression')

plt.plot(zincmlpmsedfmean['n'], zincmlpmsedfmean['l2error'],label='Multilayer Perceptron')
plt.legend(loc='best')
plt.xlabel('$\displaystyle n$',fontsize=17)
plt.xlabel('Number of data points',fontsize=17)
plt.ylabel('RMSE',fontsize=17)
plt.savefig('Overallmse.pdf')
plt.show()

In [ ]:
#generate maximum error plots of the four models

zincgaussianmaxerror = np.loadtxt("zincweaklyinformativegaussianerror.txt", comments="#", delimiter=",", unpack=False)
zincgaussianmaxerrorvague = np.loadtxt("zincuninformativegaussianerror.txt", comments="#", delimiter=",", unpack=False)

zincmapmaxerror = np.loadtxt("zincweaklyinformativemodelerror.txt", comments="#", delimiter=",", unpack=False)
zincmapmaxerrorvague = np.loadtxt("zincuninformativemodelerror.txt", comments="#", delimiter=",", unpack=False)

zincfreqmaxerror = np.loadtxt("zincridgeerror.txt", comments="#", delimiter=",", unpack=False)

zincmlpmaxerror=np.loadtxt("zincmlperror.txt", comments="#", delimiter=",", unpack=False)

zincgaussianmaxerrordf = pd.DataFrame(dict(n=zincgaussianmaxerror[:,0], maxerror=zincgaussianmaxerror[:,2]))

zincgaussianmaxerrorvaguedf = pd.DataFrame(dict(n=zincgaussianmaxerrorvague[:,0], maxerror=zincgaussianmaxerrorvague[:,2]))

zincmapmaxerrordf = pd.DataFrame(dict(n=zincmapmaxerror[:,0], maxerror=zincmapmaxerror[:,2]))

zincmapmaxerrorvaguedf = pd.DataFrame(dict(n=zincmapmaxerrorvague[:,0], maxerror=zincmapmaxerrorvague[:,2]))

zincfreqmaxerrordf = pd.DataFrame(dict(n=zincfreqmaxerror[:,0], maxerror=zincfreqmaxerror[:,2]))

zincmlpmaxerrordf = pd.DataFrame(dict(n=zincmlpmaxerror[:,0], maxerror=zincmlpmaxerror[:,2]))

plt.rcParams.update({
    "text.usetex": True,})

zincgaussianmaxerrordfmean=zincgaussianmaxerrordf.groupby(['n'],as_index=False).mean()
zincgaussianmaxerrorvaguedfmean=zincgaussianmaxerrorvaguedf.groupby(['n'],as_index=False).mean()
zincmapmaxerrordfmean=zincmapmaxerrordf.groupby(['n'],as_index=False).mean()
zincmapmaxerrorvaguedfmean=zincmapmaxerrorvaguedf.groupby(['n'],as_index=False).mean()

zincfreqmaxerrordfmean=zincfreqmaxerrordf.groupby(['n'],as_index=False).mean()

zincmlpmaxerrordfmean=zincmlpmaxerrordf.groupby(['n'],as_index=False).mean()

plt.xticks(range(0,21))

plt.plot(zincgaussianmaxerrordfmean['n'], zincgaussianmaxerrordfmean['maxerror'],label='Informative Prior, Gaussian Model')
plt.plot(zincgaussianmaxerrorvaguedfmean['n'],zincgaussianmaxerrorvaguedfmean['maxerror'],label='Vague Prior, Gaussian Model')

plt.plot(zincmapmaxerrordfmean['n'], zincmapmaxerrordfmean['maxerror'],label='Informative Prior, Proposed Model')
plt.plot(zincmapmaxerrorvaguedfmean['n'], zincmapmaxerrorvaguedfmean['maxerror'],label='Vague Prior, Proposed Model')
plt.plot(zincfreqmaxerrordfmean['n'], zincfreqmaxerrordfmean['maxerror'],label='Ridge Regression')

plt.plot(zincmlpmaxerrordfmean['n'], zincmlpmaxerrordfmean['maxerror'],label='Multilayer Perceptron')
plt.legend(loc='best')
plt.xlabel('$\displaystyle n$',fontsize=17)
plt.xlabel('Number of data points',fontsize=17)
plt.ylabel('Max error',fontsize=17)
plt.savefig('Overallmaxerror.pdf')
plt.show()

In [ ]:
#Code for calculating frequentist coverage of marginal posterior 95% HDI

pointsall=[2, 14, 4, 32, 59, 67, 48, 0, 9, 36, 18, 7, 15, 19, 27, 33, 6, 5, 57, 12]
truevalues=outputdatachildprocess['quantity'][availablechildstocksandflows]

i=10 #size of subset of data used, we used i=5,10,15,20 for results in the paper

points=pointsall[0:i]
incompletedesigncompact,incompletematrixstockscompact,incompletematrixflowscompact,incompletedatavector,incompletedataflownumber=createincompletedata(points,datamat,availablechildstocksandflows,m,N)

numberofdata=300
#generate 300 datasets from the likelihood
sampledatavector=samplefromlikelihood(incompletedatavector, incompletedataflownumber, numberofdata, m)

np.savetxt('sampledatavector'+str(i)+'data'+str(numberofdata)+'times.txt', sampledatavector,delimiter=',')


In [ ]:
#For each dataset, calculate the posterior 95% HDI and check the coverage, weakly informative prior

for times in range(0,numberofdata): 
    print('times')
    print(times)
    modelpred,trace=truncatednormalmodel(priormean,covariancevec,incompletematrixstockscompact,incompletematrixflowscompact, \
                                        sampledatavector[times,:],incompletedataflownumber,availablechildstocksandflows,m,0)

    cr_95_quantile,distancetotruth,ci_95_marginal=calculatecoverage(modelpred, trace, availablechildstocksandflows, outputdatachildprocess)

    checkcoveragetruncatednormal=[cr_95_quantile,distancetotruth,distancetotruth<cr_95_quantile,times]

    with open("checkcoverageweaklyinformativeprior"+str(i)+".txt", "a") as truncatednormalcoveragefile:
        wr = csv.writer(truncatednormalcoveragefile)
        wr.writerow(checkcoveragetruncatednormal)
        truncatednormalcoveragefile.close()

    with open("checkcoveragemarginalweaklyinformativeprior"+str(i)+".txt", "a") as truncatednormalmarginalcoveragefile:
        np.savetxt(truncatednormalmarginalcoveragefile,ci_95_marginal, delimiter=',', fmt='%s')
        truncatednormalmarginalcoveragefile.close()
        
    with open("sampleddata"+str(i)+".txt", "a") as sampleddatafile:
            wr = csv.writer(sampleddatafile)
            wr.writerow(np.array([pointsall[0:i],sampledatavector[times,:]]))
            #np.savetxt(sampleddatafile,ci_95_marginal, delimiter=',', fmt='%s')
            sampleddatafile.close()



In [ ]:
#For each dataset, calculate the posterior 95% HDI and check the coverage, uninformative prior

for times in range(0,numberofdata):
    print('times')
    print(times)
    modelpredvague,tracevague = truncatednormalmodel(priormeanvague,covariancevecvague, incompletematrixstockscompact, \
        incompletematrixflowscompact,sampledatavector[times, :], incompletedataflownumber, availablechildstocksandflows,m,0)
    
    cr_95_quantilevague,distancetotruthvague,ci_95_marginalvague=calculatecoverage(modelpredvague, tracevague, availablechildstocksandflows, outputdatachildprocess)
    
    checkcoveragetruncatednormalvague = [cr_95_quantilevague, distancetotruthvague, distancetotruthvague < cr_95_quantilevague, times]

    with open("checkcoverageuninformativeprior" + str(i) +".txt", "a") as truncatednormalcoveragefilevague:
        wr = csv.writer(truncatednormalcoveragefilevague)
        wr.writerow(checkcoveragetruncatednormalvague)
        truncatednormalcoveragefilevague.close()

    with open("checkcoveragemarginaluninformativeprior" + str(i) +".txt",
              "a") as truncatednormalmarginalcoveragefilevague:
        np.savetxt(truncatednormalmarginalcoveragefilevague, ci_95_marginalvague, delimiter=',', fmt='%s')
        truncatednormalmarginalcoveragefilevague.close()
        
    with open("sampleddatavague" + str(i) + ".txt", "a") as sampleddatafilevague:
        wr = csv.writer(sampleddatafilevague)
        wr.writerow(np.array([pointsall[0:i], sampledatavector[times, :]]))
        # np.savetxt(sampleddatafile,ci_95_marginal, delimiter=',', fmt='%s')
        sampleddatafilevague.close()


In [ ]:
#Calculates the coverage and average width of the 95% marginal posterior HDI over the 300 runs.

order=[2, 14, 4, 32, 59, 67, 48, 0, 9, 36, 18, 7, 15, 19, 27, 33, 6, 5, 57, 12]

#For weakly informative prior
coveragedata = pd.read_csv("checkcoveragemarginalweaklyinformativeprior"+str(i)+".txt",names=['Flownumber','Lower','Higher','True value','Coverage','mean']) 

#For uninformative prior
#coveragedata = pd.read_csv("checkcoveragemarginaluninformativeprior" + str(i) +".txt",names=['Flownumber','Lower','Higher','True value','Coverage','mean'])


experiments=len(coveragedata.index)/20


coveragedatasmaller=coveragedata[['Flownumber', 'Coverage']]

calculatecoverage=coveragedata.groupby(['Flownumber']).sum()
calculatecoverage['Coveragepercent']=calculatecoverage['Coverage']/experiments

print('coverage percent for each variable')
print(calculatecoverage.loc[order]['Coveragepercent'])

calculatecoverage['averagehdilength']=(calculatecoverage['Higher']-calculatecoverage['Lower'])/experiments    

print('average HDI width for each variable')
print(calculatecoverage.loc[order]['averagehdilength'])

In [ ]:
i=10